In [235]:
import os
import numpy as np
import uproot
import yaml
import re

# plot_asym helper code

In [241]:
def print_yaml_structure(data, indent=0):
    for key, value in data.items():
        print(" " * indent + key + ":")
        if isinstance(value, dict):
            print_yaml_structure(value, indent+2)
        else:
            if(type(value)==list):
                value="<LIST>"
            print(" " * (indent+2) + str(value))

In [141]:
def extract_numbers(string):
    pattern = r"\d+(?:\.\d+)?"
    numbers = re.findall(pattern, string)
    return [float(number) for number in numbers]

In [228]:
def read_splot(subdir):
    relpath = subdir.split('/')[-1]
    
    x_values = extract_numbers(relpath)
    param_names = []
    param_values = []
    param_errors = []
    # Open the TTree called ResultTree
    result_file = os.path.join(subdir, 'ResultsHSMinuit2.root')
    with uproot.open(result_file) as f:
        if 'ResultTree' in f:
            tree = f['ResultTree']

            # Loop over all branches in the tree
            for branch_name in tree.keys():

                # Skip the error branches
                if branch_name.endswith('_err'):
                    continue
                if(branch_name in ["NLL"]):
                    continue
                # Get the parameter name and its value
                param_name = branch_name
                param_value = tree[param_name].array()[0]

                # Get the corresponding error branch
                err_branch_name = f'{param_name}_err'
                if err_branch_name not in tree:
                    continue

                # Get the parameter error
                param_error = tree[err_branch_name].array()[0]

                # Append the data to the arrays
                if param_name not in param_names:
                    param_names.append(param_name)
                    param_values.append([float(param_value)])
                    param_errors.append([float(param_error)])
                else:
                    index = param_names.index(param_name)
                    param_values[index].append(float(param_value))
                    param_errors[index].append(float(param_error))
    return x_values, param_names, param_values, param_errors

In [245]:
def create_asym_yaml(project_name=""):
    path=f"/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/{project_name}/asym"
    data = {}
    nested_dict = {}
    current_dict = nested_dict
    for root, dirs, files in os.walk(path):
        for dir in dirs:
            subdir = os.path.join(root,dir)
            split_subdir = subdir.split("/")
            asym_index = split_subdir.index("asym")
            result = split_subdir[asym_index+1:]
            
            if any(element.startswith("out") for element in result) and \
               result.index(next(filter(lambda x: x.startswith("out"), result))) < len(result) - 1 and \
                ".ipynb_checkpoints" not in result:
                
                string = ""
                # Now get sPlot or Sdbnd data
                if("outObsBins_splot" in result):
                    x_values, param_names, param_values, param_errors = read_splot(subdir)
                    result[-2]="splot"
                else:
                    continue
                    result[-2]="sideband"
                data_list=result


                for i, key in enumerate(data_list[:-1]):
                    if(not key in current_dict):  
                        current_dict[key] = {}
                    if(i==len(data_list[:-1])-1): break
                    current_dict = current_dict[key]

                
                if 'x' in current_dict[key]:
                    current_dict[key]["x"].extend(x_values)
                    for p, v, e in zip(param_names, param_values, param_errors):
                        if p in current_dict[key]:
                            current_dict[key][p]["value"].extend(v)
                            current_dict[key][p]["error"].extend(e)
                        else:
                            current_dict[key][p] = {"value": v, "error": e}
                else:
                    current_dict[key] = {"x": x_values}
                    for p, v, e in zip(param_names, param_values, param_errors):
                        current_dict[key][p] = {"value": v, "error": e}
                current_dict = nested_dict
                
                # Write updated nested dictionary to the YAML file
                with open("out.yaml", "w") as file:
                    yaml.dump(nested_dict, file)
                    
    with open("out.yaml", "w") as file:
        yaml.dump(nested_dict, file)

In [247]:
def get_data_from_yaml(data, headers, par):

    current_dict = data
    for header in headers:
        current_dict = current_dict[header]
    
    
    x = current_dict["x"]
    y = current_dict[par]["value"]
    yerr = current_dict[par]["error"]

    return x, y, yerr